### Import Required Libraries

In [ ]:
import dlib
import imutils
import cv2
import time
import numpy as np
from scipy.spatial import distance as dt
from playsound import playsound
from imutils import face_utils
from threading import Thread

In [ ]:
#Function to play Alarm tune on Detecting that user is sleeping

def play_sound(path):
    playsound(path)

In [ ]:
#Calculating Aspect ratio for eye

def eye_aspect_ratio(eye):
    A=dt.euclidean(eye[1], eye[5])
    B=dt.euclidean(eye[2], eye[4])
    C=dt.euclidean(eye[0], eye[3])
    ratio=(A+B)/(2.0*C)
    return ratio

In [ ]:
#Calculating Aspect Ratio for mouth

def mouth_aspect_ratio(mouth):
    A=dt.euclidean(mouth[2], mouth[6])
    B=dt.euclidean(mouth[0], mouth[4])
    ratio=A/B
    return ratio

In [ ]:
#Data file which helps us predict Facial landmark features using dlib
predictor_file='/content/drive/My Drive/Kaggle/Projects/Drowsiness Detector/shape_predictor_68_face_landmarks.dat'

#Path to alarm tune(.mp3)
alarm='/content/drive/My Drive/Kaggle/Projects/Drowsiness Detector/alarm_clock_tonight.mp3'

In [ ]:
#Threshold for Eye aspect ratio, if Aspect ratio of eye < threshold i.e. user is sleeping, Ring the alarm
eye_ratio_thresh=0.25

#Threshold for Eye aspect ratio, if Aspect ratio of mouth > threshold i.e. user is Yawning, Alert the user to have a break
mouth_ratio_thresh=0.2

#Maximum no. of frames required for which eyes are closed, then ring the alarm (We Don't wanna ring it for an eyeblink)
eye_max_frames=10

play_alarm=False

#Flag to count no. of frames for which eye's been closed
counter=0

In [ ]:
#Detect Face
detector=dlib.get_frontal_face_detector()

#Detect facial landmark features
predictor=dlib.shape_predictor(predictor_file)

In [ ]:
#Indexes for Left, right eye and mouth from predcition array

(left_eye_start, left_eye_end)=face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(right_eye_start, right_eye_end)=face_utils.FACIAL_LANDMARKS_IDXS['right_eye']
(mouth_start, mouth_end)=face_utils.FACIAL_LANDMARKS_IDXS['inner_mouth']

In [ ]:
#Path for saving Output video

writer=None
output='/content/drive/My Drive/Kaggle/Projects/Drowsiness Detector/output.avi'

In [ ]:
video=cv2.VideoCapture(0)
while(True):
    check, frame=video.read()
    if check is False:
        break
    frame=imutils.resize(frame, width=450)
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=detector(gray, 0)
    
    #Iterate for all faces detected
    for face in faces:
        shape=predictor(gray, face)
        
        #Getting numpy array for all Facial landmark features detected
        shape=face_utils.shape_to_np(shape)
        
        #Co-ordinates for Left, right eye and mouth
        left_eye=shape[left_eye_start:left_eye_end]
        right_eye=shape[right_eye_start:right_eye_end]
        mouth=shape[mouth_start:mouth_end]
        
        #Getting eyes and mouth's aspect ratio
        left_eye_ar=eye_aspect_ratio(left_eye)
        right_eye_ar=eye_aspect_ratio(right_eye)
        average_ratio=(left_eye_ar+right_eye_ar)/2.0
        mouth_ar=mouth_aspect_ratio(mouth)
        
        #drawing a convex hull on detections
        left_eye_hull=cv2.convexHull(left_eye)
        right_eye_hull=cv2.convexHull(right_eye)
        mouth_hull=cv2.convexHull(mouth)
        cv2.drawContours(frame, [left_eye_hull], -1, (0,255,0),1)
        cv2.drawContours(frame, [right_eye_hull], -1, (0,255,0), 1)
        cv2.drawContours(frame, [mouth_hull], -1, (255,255,0),1)
        
        #Detecting sleep
        if average_ratio<eye_ratio_thresh:
            counter+=1
            if counter>eye_max_frames:
                cv2.putText(frame, 'Don\'t SLEEP!!!', (180, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,100,255),2)
                play_alarm=True
                t=Thread(target=play_sound, args=(alarm))
                t.deamon=True
                t.start()
        else:
            counter=0
            play_alarm=False
            
        #Detecting Yawning
        if mouth_ar>mouth_ratio_thresh and play_alarm==False:
            cv2.putText(frame, 'Feeling Drowsy, Have A Break!', (75, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (10,255,50),2)
        cv2.imshow('Output',frame)
        cv2.waitKey(1)
        
        #Saving frame to output video
        if writer==None and output!='':
            fourcc=cv2.VideoWriter_fourcc(*'MJPG')
            writer=cv2.VideoWriter(output, fourcc, video.get(cv2.CAP_PROP_FPS), (frame.shape[1],frame.shape[0]),True)
        if writer is not None:
            writer.write(frame)
video.release()
writer.release()